In [1]:
import numpy as np
import pandas as pd
from glob import glob

In [2]:
df_key = pd.read_csv("data/raw/key.csv", encoding='latin-1')

In [4]:
filenames = glob("data/raw/BP_*_00A1.csv")

In [5]:
dataframes = [pd.read_csv(f, encoding='latin-1') for f in filenames]

In [175]:
for dataframe in dataframes:
    newcols = []
    for column in dataframe.columns:
        column = column.replace(".","").replace("-","").replace("_","")
        newcols.append(column)
    dataframe.columns = newcols   
  #  print(dataframe.columns)
    

In [154]:
dataframes2 =[]
for dataframe in dataframes:
    dataframe = dataframe[["GEOid2",
                           "GEOdisplaylabel",
                           "GEOfipsstatecode",
                           "GEOfipscountycode",
                           "NAICSdisplaylabel",
                           "ESTAB",
                           "EMP",
                           "YEARid"]]
    dataframe.columns = ["geo_id",
                       "county_name",
                       "state_fips",
                       "county_fips",
                       "naics_industry",
                       "establishments",
                       "employees",
                       "year"]
    dataframe = dataframe.set_index(["geo_id"])
    dataframe = dataframe[dataframe.county_fips != 999]
    dataframe = dataframe[dataframe.naics_industry != "Total for all sectors"]
    dataframe.naics_industry = dataframe.naics_industry.str.replace("Mining","Mining, quarrying, and oil and gas extraction")

    dataframes2.append(dataframe)

In [155]:
stacked = pd.concat(dataframes2)

In [176]:
stacked.loc[6003].groupby("year").sum()

,state_fips,county_fips,establishments,employees
year,,,,
2005,90,45,61,693.0
2006,90,45,58,709.0
2007,72,36,44,41.0
2008,78,39,45,55.0
2009,90,45,43,32.0
2010,90,45,42,34.0
2011,84,42,46,627.0
2012,84,42,43,25.0
2013,84,42,36,72.0


In [174]:
stacked.tail()

,county_name,state_fips,county_fips,naics_industry,establishments,employees,year
geo_id,,,,,,,
56045,"Weston County, Wyoming",56,45,Health care and social assistance,22,364.0,2015
56045,"Weston County, Wyoming",56,45,"Arts, entertainment, and recreation",2,NaN,2015
56045,"Weston County, Wyoming",56,45,Accommodation and food services,25,175.0,2015
56045,"Weston County, Wyoming",56,45,Other services (except public administration),18,66.0,2015
56045,"Weston County, Wyoming",56,45,Industries not classified,1,NaN,2015
